# Imports and preprocessing

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Softmax
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCELoss
from torch.nn import CrossEntropyLoss
import matplotlib.pyplot as plt
from torch.nn import functional as F
import time


import os
from os import path
from pathlib import Path

import pickle
import time
import warnings
warnings.filterwarnings('ignore')

### Data preprocessing

In [ ]:
DP_DIR = "Data_preprocessing"
export_dir = Path(os.getcwd())
files_path = Path(export_dir.parent, DP_DIR)

In [ ]:
train_data_mixed = pd.read_csv(Path(files_path,'train_data.csv'))
test_data = pd.read_csv(Path(files_path,'test_data.csv'))
train_array = train_data_mixed.to_numpy()
test_array = test_data.to_numpy()

In [ ]:
with open(Path(files_path,'items_values_dict.pkl'), 'rb') as f:
    items_values_dict = pickle.load(f) # deserialize using load()

In [ ]:
train_array = train_data_mixed.iloc[:,:-3].to_numpy()
test_array = test_data.iloc[:,:-2].to_numpy()
test_user_ids = test_data.iloc[:,-2].to_numpy()
test_y_pos = test_data['y_positive'].to_numpy()

test_array_without_item = test_array.copy()
for i in range(len(test_array)):
    test_array_without_item[i][test_y_pos[i]] = 0 

In [ ]:
num_users = 6040
num_items = 3706

In [ ]:
softmax = nn.Softmax()

In [ ]:
with open(Path(files_path,'prob_dict.pkl'), 'rb') as f:
    prob_dict = pickle.load(f)
    
items_values= pd.read_csv(Path(files_path,'items_values.csv'))

items_array = items_values.to_numpy()

In [ ]:
sorted_prob_dict  = list(sorted(prob_dict.items(), key=lambda item: item[1],reverse=True))

In [ ]:
pop_index_dict = {} # dictionary where the key is the item id and the value is the popularity of the item
for i in range(len(sorted_prob_dict)):
    pop_index_dict[sorted_prob_dict[i][0]] = i+1

In [ ]:
with open(Path(files_path,'user_similarities_Jaccard.pkl'), 'rb') as f:
    user_based_Jaccard_sim = pickle.load(f) # deserialize using load()
    

In [ ]:
with open(Path(files_path,'cosine_items.pkl'), 'rb') as f:
    cosine_items = pickle.load(f) # deserialize using load()

In [ ]:
file_path = 'tf_idf_items.pkl'
with open(Path(files_path,file_path), 'rb') as f:
    tf_idf_items = pickle.load(f)  

In [ ]:
file_path = 'shap_values.pkl'
with open(Path(files_path,file_path), 'rb') as f:
    shap_values = pickle.load(f)

file_path = 'item_to_cluster.pkl'
with open(Path(files_path,file_path), 'rb') as f:
    item_to_cluster = pickle.load(f)

# VAE Model definition and training

In [ ]:
import ipynb
import importlib
from ipynb.fs.defs.VAE import MultVAE
importlib.reload(ipynb.fs.defs.VAE)
from ipynb.fs.defs.VAE import MultVAE

In [ ]:
config= {
    "data_name": "ml-1m",
    "train_ratio":0.8,
  
    "enc_dims": [512,128],
    "dropout": 0.5,
    "anneal_cap": 0.2,
    "total_anneal_steps": 200000,
  
    "num_epochs": 500,
    "batch_size": 512,
    "test_batch_size": 512,
    "learning_rate": 0.01,
    "early_stop": True,
    "patience": 50,
  
    "top_k": [100]
  }


# Train the model on GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("num_users is ", num_users)
print("num_items is ", num_items)
VAE_recommender = MultVAE(config, num_users,num_items, device)

In [ ]:
checkpoint = torch.load("VAE_epoch_9.pt")
VAE_recommender.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
for param in VAE_recommender.parameters():
    param.requires_grad= False

# Import LXR 

In [ ]:
import ipynb
import importlib
from ipynb.fs.defs.LXR_VAE_model import MLP_G, Recommender_G, Explainer_G, LossModelCombined
importlib.reload(ipynb.fs.defs.LXR_VAE_model)
from ipynb.fs.defs.LXR_VAE_model import MLP_G, Recommender_G, Explainer_G, LossModelCombined

In [ ]:
hidden_dim=20
backbone = Recommender_G(num_items, hidden_dim, device)
backbone.load_state_dict(torch.load("MLP_backbone.pt"))
backbone.eval()

### Recommender Freezing

In [ ]:
for param in backbone.parameters():
    param.requires_grad = False

In [ ]:
alpha_parameter = 0.008
explainer_model_g = Explainer_G(backbone, num_items, hidden_dim, device).to(device)
explainer_path = "LXR_VAE_explainer_0.00125_11.pt"
explainer_checkpoint = torch.load(explainer_path)
explainer_model_g.load_state_dict(explainer_checkpoint['model_state_dict'])

In [ ]:
modelCombined_g = LossModelCombined(alpha_parameter, backbone, VAE_recommender, explainer_model_g, hidden_dim, device).to(device)
lxr_path = "LXR_VAE_0.00125_11.pt"
lxr_checkpoint = torch.load(lxr_path)
modelCombined_g.load_state_dict(lxr_checkpoint['model_state_dict'])

In [ ]:
for param in modelCombined_g.parameters():
    param.requires_grad = False

In [ ]:
for param in explainer_model_g.parameters():
    param.requires_grad = False

# Init LIME

In [ ]:
import ipynb
from ipynb.fs.defs.lime import distance_to_proximity, LimeBase, get_lime_args, gaussian_kernel
importlib.reload(ipynb.fs.defs.lime)
from ipynb.fs.defs.lime import distance_to_proximity, LimeBase, get_lime_args, gaussian_kernel
'''
distance_to_proximity(distances_list) - takes distances from origin user and returns proximity
LimeBase() - class that gets kernel function
get_lime_args(user_vetor, item_id, model, min_pert = 10, max_pert = 20, num_of_perturbations = 5, seed = 0) - 
    returns neighborhood_data, neighborhood_labels, distances, item_id
'''

lime = LimeBase(distance_to_proximity)

# Mask functions

#### LXR based similarity

In [ ]:
def find_LXR_mask(x, item_id, item_tensor, modelCombined_g):
    user_hist = x
    user_hist[item_id] = 0
    x_masked_g, loss,_,_ = modelCombined_g(user_hist, item_tensor, item_id, device)
    item_sim_dict = {i: x_masked_g[i].item() for i in range(x_masked_g.numel())}    

    return item_sim_dict

#### User based similarities using Jaccard

In [ ]:
def find_jaccard_u_mask(x, item_id, num_items, user_based_Jaccard_sim):
    user_hist = x # remove the positive item we want to explain from the user history
    user_hist[item_id] = 0
    item_jaccard_dict = {}
    for i in range(num_items): 
        if(user_hist[i] == 1): # iterate all positive items of the user
            if((i,item_id) in user_based_Jaccard_sim.keys()):
                item_jaccard_dict[i]=user_based_Jaccard_sim[(i,item_id)] # add Jaccard similarity between items
            else:
                item_jaccard_dict[i] = 0
            
    return item_jaccard_dict

#### Cosine based similarities between users and items

In [ ]:
def find_cosine_mask(x, item_id, num_items, item_cosine):
    user_hist = x # remove the positive item we want to explain from the user history
    user_hist[item_id] = 0
    item_cosine_dict = {}
        
    for i in range(num_items): 
        if(user_hist[i] == 1): # iterate all positive items of the user
            if((i,item_id) in item_cosine.keys()):
                item_cosine_dict[i]=item_cosine[(i,item_id)] # add cosine similarity between items
            else:
                item_cosine_dict[i] = 0
    
    return item_cosine_dict

#### Popularity

In [ ]:

def find_pop_mask(x, item_id, num_items):
    user_hist = torch.tensor(x) # remove the positive item we want to explain from the user history
    user_hist[item_id] = 0
    item_pop_dict = {}
    for i in range(num_items): 
        if(user_hist[i] == 1): # iterate over all positive items of the user
            item_pop_dict[i]=prob_dict[i] # add the pop of the item to the dictionary
            
            
    return item_pop_dict

#### Lime

In [ ]:
def find_lime_mask(x, item_id, min_pert, max_pert, num_of_perturbations, kernel_func, feature_selection, recommender, num_samples=10, method = 'POS'):
    user_hist = x # remove the positive item we want to explain from the user history
    user_hist[item_id] = 0
    lime.kernel_fn = kernel_func
    neighborhood_data, neighborhood_labels, distances, item_id = get_lime_args(user_hist, item_id, recommender, min_pert = min_pert, max_pert = max_pert, num_of_perturbations = num_of_perturbations, seed = item_id)
    if method=='POS':
        most_pop_items  = lime.explain_instance_with_data(neighborhood_data, neighborhood_labels, distances, item_id, num_samples, feature_selection, pos_neg='POS')
    if method=='NEG':
        most_pop_items  = lime.explain_instance_with_data(neighborhood_data, neighborhood_labels, distances, item_id, num_samples, feature_selection ,pos_neg='NEG')
        
    return most_pop_items 

#### TF-IDF

In [ ]:
def find_tf_idf_mask(x, item_id, num_items, tf_idf_sim):
    x = x.cpu().detach().numpy()
    x[item_id] = 0
   
    positive_items = np.where(x == 1)[0]
    tf_idf_dict = {i: tf_idf_sim.get((i, item_id), 0) for i in positive_items}
   
    return tf_idf_dict

#### SHAPE

In [ ]:
def find_shapley_mask(user_vector, user_id, model, shap_values, item_to_cluster):
        item_shap = {}
        shapley_values = shap_values[shap_values[:, 0].astype(int) == user_id][:,1:].flatten()
        user_vector = user_vector.cpu().detach().numpy().astype(int)
        for i in np.where(user_vector.astype(int) == 1)[0]:
            items_cluster = item_to_cluster[i]
            item_shap[i] = shapley_values[items_cluster]
        return item_shap

# Utils functions for evaluations

In [ ]:
def VAE_get_top_k(user_tensor, original_user_vector, num_items, model, top_k):
    item_prob_dict = {}
    output_model = [float(i) for i in softmax(model(user_tensor)[0]).cpu().detach().numpy()]
    original_user_vector = np.array(original_user_vector.cpu())
    neg = np.ones_like(original_user_vector)- original_user_vector
    output = neg*output_model
    for i in range(len(output)):
        if output[i] > 0:
            item_prob_dict[i]=output[i]
    sorted_items_by_prob  = sorted(item_prob_dict.items(), key=lambda item: item[1],reverse=True)
    top_k = min(top_k, len(sorted_items_by_prob))
    return dict(sorted_items_by_prob[0:top_k])

In [ ]:
def VAE_get_index_in_the_list(user_tensor, item_id, num_items, original_user_tensor, recommender):
    return list(VAE_get_top_k(user_tensor, original_user_tensor, num_items, recommender, num_items).keys()).index(item_id)

In [ ]:
def get_user_recommended_item(user_tensor, recommender):
    user_res = softmax(recommender(user_tensor)[0])
    user_catalog = torch.ones_like(user_tensor)-user_tensor
    user_recommenations = torch.mul(user_res, user_catalog)
    return(torch.argmax(user_recommenations))

In [ ]:
def get_ndcg(ranked_list, target_item):
    if target_item not in ranked_list:
        return 0.0

    target_idx = torch.tensor(ranked_list.index(target_item), device=device)
    dcg = torch.reciprocal(torch.log2(target_idx + 2))

    return dcg.item()

In [ ]:
full_train_array = train_data_mixed.to_numpy()
full_test_array = test_data.to_numpy()

topk_train = {}
for i in range(len(full_train_array)):
    vec = full_train_array[i][:-3]
    user_id = full_train_array[i][-3]
    tens = torch.Tensor(vec).to(device)
    topk_train[user_id] = int(get_user_recommended_item(tens, VAE_recommender).cpu().detach().numpy())
    
topk_test = {}
for i in range(len(full_test_array)):
    vec = full_test_array[i][:-2]
    user_id = full_test_array[i][-2]
    tens = torch.Tensor(vec).to(device)
    topk_test[user_id] = int(get_user_recommended_item(tens, VAE_recommender).cpu().detach().numpy())

In [ ]:
def single_user_metrics(user_vector, item_id, num_of_bins, num_items, recommender_model, model_combined, user_id=None,
                        mask_type=None):
    """
    calculates all the metrics for a single user and a single masking method
    """
    user_tensor = torch.Tensor(user_vector).to(device)
    POS_masked = user_tensor
    NEG_masked = user_tensor
    POS_masked[item_id] = 0
    NEG_masked[item_id] = 0
    item_vector = items_array[item_id]
    item_tensor = torch.FloatTensor(item_vector).to(device)
    user_hist_size = np.sum(user_vector)

    bins = [0] + [len(x) for x in np.array_split(np.arange(user_hist_size), num_of_bins, axis=0)]

    POS_at_1 = [0] * (len(bins))
    POS_at_5 = [0] * (len(bins))
    POS_at_10 = [0] * (len(bins))
    POS_at_20 = [0] * (len(bins))
    POS_at_50 = [0] * (len(bins))
    POS_at_100 = [0] * (len(bins))

    NEG_at_1 = [0] * (len(bins))
    NEG_at_5 = [0] * (len(bins))
    NEG_at_10 = [0] * (len(bins))
    NEG_at_20 = [0] * (len(bins))
    NEG_at_50 = [0] * (len(bins))
    NEG_at_100 = [0] * (len(bins))

    DEL = [0] * (len(bins))
    INS = [0] * (len(bins))

    rankA_at_1 = [0] * (len(bins))
    rankA_at_5 = [0] * (len(bins))
    rankA_at_10 = [0] * (len(bins))
    rankA_at_20 = [0] * (len(bins))
    rankA_at_50 = [0] * (len(bins))
    rankA_at_100 = [0] * (len(bins))

    rankB = [0] * (len(bins))

    NDCG_at_1 = [0] * (len(bins))
    NDCG_at_5 = [0] * (len(bins))
    NDCG_at_10 = [0] * (len(bins))
    NDCG_at_20 = [0] * (len(bins))
    NDCG_at_50 = [0] * (len(bins))
    NDCG_at_100 = [0] * (len(bins))

    if mask_type == 'lime':
        POS_sim_items = find_lime_mask(user_vector, item_id, 50, 100, 400, distance_to_proximity, 'highest_weights',
                                       recommender_model, num_samples=user_hist_size)
        NEG_sim_items = find_lime_mask(user_vector, item_id, 50, 100, 400, distance_to_proximity, 'highest_weights',
                                       recommender_model, num_samples=user_hist_size, method='NEG')

    else:
        if mask_type == 'jaccard_u':
            sim_items = find_jaccard_u_mask(user_tensor, item_id, num_items, user_based_Jaccard_sim)
        elif mask_type == 'cosine':
            sim_items = find_cosine_mask(user_tensor, item_id, num_items, cosine_items)
        elif mask_type == 'pop':
            sim_items = find_pop_mask(user_tensor, item_id, num_items)
        elif mask_type == 'lxr':
            sim_items = find_LXR_mask(user_tensor, item_id, item_tensor, model_combined)
        elif mask_type == 'tf_idf':
            sim_items = find_tf_idf_mask(user_tensor, item_id, num_items, tf_idf_items)
        elif mask_type == 'shap':
            sim_items = find_shapley_mask(user_tensor, user_id, recommender_model, shap_values, item_to_cluster)

        POS_sim_items = list(sorted(sim_items.items(), key=lambda item: item[1], reverse=True))[0:user_hist_size]
        NEG_sim_items = list(sorted(dict(POS_sim_items).items(), key=lambda item: item[1], reverse=False))

    total_items = 0
    for i in range(len(bins)):
        total_items += bins[i]

        POS_masked = torch.zeros_like(user_tensor, dtype=torch.float32, device=device)
        for j in POS_sim_items[:total_items]:
            POS_masked[j[0]] = 1
        POS_masked = user_tensor - POS_masked  # remove the masked items from the user history

        NEG_masked = torch.zeros_like(user_tensor, dtype=torch.float32, device=device)
        for j in NEG_sim_items[:total_items]:
            NEG_masked[j[0]] = 1
        NEG_masked = user_tensor - NEG_masked  # remove the masked items from the user history 

        POS_ranked_list = VAE_get_top_k(POS_masked, user_tensor, num_items, recommender_model, num_items)
        POS_index = list(POS_ranked_list.keys()).index(item_id) + 1
        NEG_index = VAE_get_index_in_the_list(NEG_masked, item_id, num_items, user_tensor, recommender_model) + 1

        # for pos:
        POS_at_1[i] = 1 if POS_index <= 1 else 0
        POS_at_5[i] = 1 if POS_index <= 5 else 0
        POS_at_10[i] = 1 if POS_index <= 10 else 0
        POS_at_20[i] = 1 if POS_index <= 20 else 0
        POS_at_50[i] = 1 if POS_index <= 50 else 0
        POS_at_100[i] = 1 if POS_index <= 100 else 0

        # for neg:
        NEG_at_1[i] = 1 if NEG_index <= 1 else 0
        NEG_at_5[i] = 1 if NEG_index <= 5 else 0
        NEG_at_10[i] = 1 if NEG_index <= 10 else 0
        NEG_at_20[i] = 1 if NEG_index <= 20 else 0
        NEG_at_50[i] = 1 if NEG_index <= 50 else 0
        NEG_at_100[i] = 1 if NEG_index <= 100 else 0

        # for del:
        DEL[i] = float(softmax(recommender_model(POS_masked)[0])[item_id].detach().cpu().numpy())

        # for ins:
        INS[i] = float(softmax(recommender_model(user_tensor - POS_masked)[0])[item_id].detach().cpu().numpy())

        # for rankA:
        rankA_at_1[i] = max(0, (1 + 1 - POS_index) / 10)
        rankA_at_5[i] = max(0, (5 + 1 - POS_index) / 20)
        rankA_at_10[i] = max(0, (10 + 1 - POS_index) / 10)
        rankA_at_20[i] = max(0, (20 + 1 - POS_index) / 20)
        rankA_at_50[i] = max(0, (50 + 1 - POS_index) / 50)
        rankA_at_100[i] = max(0, (100 + 1 - POS_index) / 100)

        # for rankB:
        rankB[i] = 1 / POS_index

        # for NDCG:
        NDCG_at_1[i] = get_ndcg(list(POS_ranked_list.keys())[:1], item_id)
        NDCG_at_5[i] = get_ndcg(list(POS_ranked_list.keys())[:5], item_id)
        NDCG_at_10[i] = get_ndcg(list(POS_ranked_list.keys())[:10], item_id)
        NDCG_at_20[i] = get_ndcg(list(POS_ranked_list.keys())[:20], item_id)
        NDCG_at_50[i] = get_ndcg(list(POS_ranked_list.keys())[:50], item_id)
        NDCG_at_100[i] = get_ndcg(list(POS_ranked_list.keys())[:100], item_id)

    res = [POS_at_1, POS_at_5, POS_at_10, POS_at_20, POS_at_50, POS_at_100, NEG_at_1, NEG_at_5, NEG_at_10, NEG_at_20,
           NEG_at_50, NEG_at_100, DEL, INS, rankA_at_1, rankA_at_5, rankA_at_10, rankA_at_20, rankA_at_50, rankA_at_100,
           rankB, NDCG_at_1, NDCG_at_5, NDCG_at_10, NDCG_at_20, NDCG_at_50, NDCG_at_100]
    for i in range(len(res)):
        res[i] = np.array(res[i])

    return res

# Evaluate on all baselines

In [ ]:
VAE_recommender.eval()
modelCombined_g.eval()

# Evaluate the model on the test set
loss_fn = nn.BCELoss()
test_losses = []
y_true = []
y_pred = []
correct = 0
threshold = 0.5
total = len(test_data)
num_of_bins = 11
k = 10

POS_at_1_j_u = np.zeros(num_of_bins)
pos_at_5_j_u = np.zeros(num_of_bins)
POS_at_10_j_u = np.zeros(num_of_bins)
POS_at_20_j_u = np.zeros(num_of_bins)
POS_at_50_j_u = np.zeros(num_of_bins)
POS_at_100_j_u = np.zeros(num_of_bins)
NEG_at_1_j_u = np.zeros(num_of_bins)
NEG_at_5_j_u = np.zeros(num_of_bins)
NEG_at_10_j_u = np.zeros(num_of_bins)
NEG_at_20_j_u = np.zeros(num_of_bins)
NEG_at_50_j_u = np.zeros(num_of_bins)
NEG_at_100_j_u = np.zeros(num_of_bins)
users_DEL_j_u = np.zeros(num_of_bins)
users_INS_j_u = np.zeros(num_of_bins)
rank_at_1_j_u = np.zeros(num_of_bins)
rank_at_5_j_u = np.zeros(num_of_bins)
rank_at_10_A_j_u = np.zeros(num_of_bins)
rank_at_20_A_j_u = np.zeros(num_of_bins)
rank_at_50_A_j_u = np.zeros(num_of_bins)
rank_at_100_A_j_u = np.zeros(num_of_bins)
rank_at_k_B_j_u = np.zeros(num_of_bins)
NDCG_at_1_j_u = np.zeros(num_of_bins)
NDCG_at_5_j_u = np.zeros(num_of_bins)
NDCG_at_10_j_u = np.zeros(num_of_bins)
NDCG_at_20_j_u = np.zeros(num_of_bins)
NDCG_at_50_j_u = np.zeros(num_of_bins)
NDCG_at_100_j_u = np.zeros(num_of_bins)


POS_at_1_c_s = np.zeros(num_of_bins)
pos_at_5_c_s = np.zeros(num_of_bins)
POS_at_10_c_s = np.zeros(num_of_bins)
POS_at_20_c_s = np.zeros(num_of_bins)
POS_at_50_c_s = np.zeros(num_of_bins)
POS_at_100_c_s = np.zeros(num_of_bins)
NEG_at_1_c_s = np.zeros(num_of_bins)
NEG_at_5_c_s = np.zeros(num_of_bins)
NEG_at_10_c_s = np.zeros(num_of_bins)
NEG_at_20_c_s= np.zeros(num_of_bins)
NEG_at_50_c_s = np.zeros(num_of_bins)
NEG_at_100_c_s = np.zeros(num_of_bins)
users_DEL_c_s = np.zeros(num_of_bins)
users_INS_c_s = np.zeros(num_of_bins)
rank_at_1_c_s = np.zeros(num_of_bins)
rank_at_5_c_s = np.zeros(num_of_bins)
rank_at_10_A_c_s = np.zeros(num_of_bins)
rank_at_20_A_c_s = np.zeros(num_of_bins)
rank_at_50_A_c_s = np.zeros(num_of_bins)
rank_at_100_A_c_s = np.zeros(num_of_bins)
rank_at_k_B_c_s = np.zeros(num_of_bins)
NDCG_at_1_c_s = np.zeros(num_of_bins)
NDCG_at_5_c_s = np.zeros(num_of_bins)
NDCG_at_10_c_s = np.zeros(num_of_bins)
NDCG_at_20_c_s = np.zeros(num_of_bins)
NDCG_at_50_c_s = np.zeros(num_of_bins)
NDCG_at_100_c_s = np.zeros(num_of_bins)


POS_at_1_pop = np.zeros(num_of_bins)
pos_at_5_pop = np.zeros(num_of_bins)
POS_at_10_pop = np.zeros(num_of_bins)
POS_at_20_pop = np.zeros(num_of_bins)
POS_at_50_pop = np.zeros(num_of_bins)
POS_at_100_pop = np.zeros(num_of_bins)
NEG_at_1_pop = np.zeros(num_of_bins)
NEG_at_5_pop = np.zeros(num_of_bins)
NEG_at_10_pop = np.zeros(num_of_bins)
NEG_at_20_pop = np.zeros(num_of_bins)
NEG_at_50_pop = np.zeros(num_of_bins)
NEG_at_100_pop = np.zeros(num_of_bins)
users_DEL_pop = np.zeros(num_of_bins)
users_INS_pop = np.zeros(num_of_bins)
rank_at_1_pop = np.zeros(num_of_bins)
rank_at_5_pop = np.zeros(num_of_bins)
rank_at_10_A_pop = np.zeros(num_of_bins)
rank_at_20_A_pop = np.zeros(num_of_bins)
rank_at_50_A_pop = np.zeros(num_of_bins)
rank_at_100_A_pop = np.zeros(num_of_bins)
rank_at_k_B_pop = np.zeros(num_of_bins)
NDCG_at_1_pop = np.zeros(num_of_bins)
NDCG_at_5_pop = np.zeros(num_of_bins)
NDCG_at_10_pop = np.zeros(num_of_bins)
NDCG_at_20_pop = np.zeros(num_of_bins)
NDCG_at_50_pop = np.zeros(num_of_bins)
NDCG_at_100_pop = np.zeros(num_of_bins)

POS_at_1_lime = np.zeros(num_of_bins)
pos_at_5_lime = np.zeros(num_of_bins)
POS_at_10_lime = np.zeros(num_of_bins)
POS_at_20_lime = np.zeros(num_of_bins)
POS_at_50_lime = np.zeros(num_of_bins)
POS_at_100_lime = np.zeros(num_of_bins)
NEG_at_1_lime = np.zeros(num_of_bins)
NEG_at_5_lime = np.zeros(num_of_bins)
NEG_at_10_lime = np.zeros(num_of_bins)
NEG_at_20_lime = np.zeros(num_of_bins)
NEG_at_50_lime = np.zeros(num_of_bins)
NEG_at_100_lime = np.zeros(num_of_bins)
users_DEL_lime = np.zeros(num_of_bins)
users_INS_lime = np.zeros(num_of_bins)
rank_at_1_lime = np.zeros(num_of_bins)
rank_at_5_lime = np.zeros(num_of_bins)
rank_at_10_A_lime = np.zeros(num_of_bins)
rank_at_20_A_lime = np.zeros(num_of_bins)
rank_at_50_A_lime = np.zeros(num_of_bins)
rank_at_100_A_lime = np.zeros(num_of_bins)
rank_at_k_B_lime = np.zeros(num_of_bins)
NDCG_at_1_lime = np.zeros(num_of_bins)
NDCG_at_5_lime = np.zeros(num_of_bins)
NDCG_at_10_lime = np.zeros(num_of_bins)
NDCG_at_20_lime = np.zeros(num_of_bins)
NDCG_at_50_lime = np.zeros(num_of_bins)
NDCG_at_100_lime = np.zeros(num_of_bins)

POS_at_1_tf_idf = np.zeros(num_of_bins)
pos_at_5_tf_idf = np.zeros(num_of_bins)
POS_at_10_tf_idf = np.zeros(num_of_bins)
POS_at_20_tf_idf = np.zeros(num_of_bins)
POS_at_50_tf_idf = np.zeros(num_of_bins)
POS_at_100_tf_idf = np.zeros(num_of_bins)
NEG_at_1_tf_idf = np.zeros(num_of_bins)
NEG_at_5_tf_idf = np.zeros(num_of_bins)
NEG_at_10_tf_idf = np.zeros(num_of_bins)
NEG_at_20_tf_idf = np.zeros(num_of_bins)
NEG_at_50_tf_idf = np.zeros(num_of_bins)
NEG_at_100_tf_idf = np.zeros(num_of_bins)
users_DEL_tf_idf = np.zeros(num_of_bins)
users_INS_tf_idf = np.zeros(num_of_bins)
rank_at_1_tf_idf = np.zeros(num_of_bins)
rank_at_5_tf_idf = np.zeros(num_of_bins)
rank_at_10_A_tf_idf = np.zeros(num_of_bins)
rank_at_20_A_tf_idf = np.zeros(num_of_bins)
rank_at_50_A_tf_idf = np.zeros(num_of_bins)
rank_at_100_A_tf_idf = np.zeros(num_of_bins)
rank_at_k_B_tf_idf = np.zeros(num_of_bins)
NDCG_at_1_tf_idf = np.zeros(num_of_bins)
NDCG_at_5_tf_idf = np.zeros(num_of_bins)
NDCG_at_10_tf_idf = np.zeros(num_of_bins)
NDCG_at_20_tf_idf = np.zeros(num_of_bins)
NDCG_at_50_tf_idf = np.zeros(num_of_bins)
NDCG_at_100_tf_idf = np.zeros(num_of_bins)

POS_at_1_shap = np.zeros(num_of_bins)
pos_at_5_shap = np.zeros(num_of_bins)
POS_at_10_shap = np.zeros(num_of_bins)
POS_at_20_shap = np.zeros(num_of_bins)
POS_at_50_shap = np.zeros(num_of_bins)
POS_at_100_shap = np.zeros(num_of_bins)
NEG_at_1_shap = np.zeros(num_of_bins)
NEG_at_5_shap = np.zeros(num_of_bins)
NEG_at_10_shap = np.zeros(num_of_bins)
NEG_at_20_shap = np.zeros(num_of_bins)
NEG_at_50_shap = np.zeros(num_of_bins)
NEG_at_100_shap = np.zeros(num_of_bins)
users_DEL_shap = np.zeros(num_of_bins)
users_INS_shap = np.zeros(num_of_bins)
rank_at_1_shap = np.zeros(num_of_bins)
rank_at_5_shap = np.zeros(num_of_bins)
rank_at_10_A_shap = np.zeros(num_of_bins)
rank_at_20_A_shap = np.zeros(num_of_bins)
rank_at_50_A_shap = np.zeros(num_of_bins)
rank_at_100_A_shap = np.zeros(num_of_bins)
rank_at_k_B_shap = np.zeros(num_of_bins)
NDCG_at_1_shap = np.zeros(num_of_bins)
NDCG_at_5_shap = np.zeros(num_of_bins)
NDCG_at_10_shap = np.zeros(num_of_bins)
NDCG_at_20_shap = np.zeros(num_of_bins)
NDCG_at_50_shap = np.zeros(num_of_bins)
NDCG_at_100_shap = np.zeros(num_of_bins)

POS_at_1_lxr = np.zeros(num_of_bins)
pos_at_5_lxr = np.zeros(num_of_bins)
POS_at_10_lxr = np.zeros(num_of_bins)
POS_at_20_lxr = np.zeros(num_of_bins)
POS_at_50_lxr = np.zeros(num_of_bins)
POS_at_100_lxr = np.zeros(num_of_bins)
NEG_at_1_lxr = np.zeros(num_of_bins)
NEG_at_5_lxr = np.zeros(num_of_bins)
NEG_at_10_lxr = np.zeros(num_of_bins)
NEG_at_20_lxr = np.zeros(num_of_bins)
NEG_at_50_lxr = np.zeros(num_of_bins)
NEG_at_100_lxr = np.zeros(num_of_bins)
users_DEL_lxr = np.zeros(num_of_bins)
users_INS_lxr = np.zeros(num_of_bins)
rank_at_1_lxr = np.zeros(num_of_bins)
rank_at_5_lxr = np.zeros(num_of_bins)
rank_at_10_A_lxr = np.zeros(num_of_bins)
rank_at_20_A_lxr = np.zeros(num_of_bins)
rank_at_50_A_lxr = np.zeros(num_of_bins)
rank_at_100_A_lxr = np.zeros(num_of_bins)
rank_at_k_B_lxr = np.zeros(num_of_bins)
NDCG_at_1_lxr = np.zeros(num_of_bins)
NDCG_at_5_lxr = np.zeros(num_of_bins)
NDCG_at_10_lxr = np.zeros(num_of_bins)
NDCG_at_20_lxr = np.zeros(num_of_bins)
NDCG_at_50_lxr = np.zeros(num_of_bins)
NDCG_at_100_lxr = np.zeros(num_of_bins)


num_of_bins = 10

with torch.no_grad():
    for i in range(test_array.shape[0]):
        if i % 500 == 0:
            print(i)
        start_time = time.time()
        user_vector = test_array[i]
        user_tensor = torch.FloatTensor(user_vector).to(device)
        user_id = test_user_ids[i]
        item_id = int(get_user_recommended_item(user_tensor, VAE_recommender).detach().cpu().numpy())
        item_vector =  items_values_dict[item_id]
        item_tensor = torch.FloatTensor(item_vector).to(device)

        user_vector[item_id] = 0
        user_tensor [item_id] = 0
        
        res = single_user_metrics(user_vector, item_id, num_of_bins, num_items, VAE_recommender, modelCombined_g, mask_type= 'jaccard_u')
        POS_at_1_j_u += res[0]
        pos_at_5_j_u += res[1]
        POS_at_10_j_u += res[2]
        POS_at_20_j_u += res[3]
        POS_at_50_j_u += res[4]
        POS_at_100_j_u += res[5]
        NEG_at_1_j_u += res[6]
        NEG_at_5_j_u += res[7]
        NEG_at_10_j_u += res[8]
        NEG_at_20_j_u += res[9]
        NEG_at_50_j_u += res[10]
        NEG_at_100_j_u += res[11]
        users_DEL_j_u += res[12]
        users_INS_j_u += res[13]
        rank_at_1_j_u += res[14]
        rank_at_5_j_u += res[15]
        rank_at_10_A_j_u += res[16]
        rank_at_20_A_j_u += res[17]
        rank_at_50_A_j_u += res[18]
        rank_at_100_A_j_u += res[19]
        rank_at_k_B_j_u += res[20]
        NDCG_at_1_j_u += res[21]
        NDCG_at_5_j_u += res[22]
        NDCG_at_10_j_u += res[23]
        NDCG_at_20_j_u += res[24]
        NDCG_at_50_j_u += res[25]
        NDCG_at_100_j_u += res[26]


        res = single_user_metrics(user_vector, item_id, num_of_bins, num_items, VAE_recommender, modelCombined_g, mask_type= 'cosine')
        POS_at_1_c_s += res[0]
        pos_at_5_c_s += res[1]
        POS_at_10_c_s += res[2]
        POS_at_20_c_s += res[3]
        POS_at_50_c_s += res[4]
        POS_at_100_c_s += res[5]
        NEG_at_1_c_s += res[6]
        NEG_at_5_c_s += res[7]
        NEG_at_10_c_s += res[8]
        NEG_at_20_c_s += res[9]
        NEG_at_50_c_s += res[10]
        NEG_at_100_c_s += res[11]
        users_DEL_c_s += res[12]
        users_INS_c_s += res[13]
        rank_at_1_c_s += res[14]
        rank_at_5_c_s += res[15]
        rank_at_10_A_c_s += res[16]
        rank_at_20_A_c_s += res[17]
        rank_at_50_A_c_s += res[18]
        rank_at_100_A_c_s += res[19]
        rank_at_k_B_c_s += res[20]
        NDCG_at_1_c_s += res[21]
        NDCG_at_5_c_s += res[22]
        NDCG_at_10_c_s += res[23]
        NDCG_at_20_c_s += res[24]
        NDCG_at_50_c_s += res[25]
        NDCG_at_100_c_s += res[26]

        res = single_user_metrics(user_vector, item_id, num_of_bins, num_items, VAE_recommender, modelCombined_g, mask_type= 'pop')
        POS_at_1_pop += res[0]
        pos_at_5_pop += res[1]
        POS_at_10_pop += res[2]
        POS_at_20_pop += res[3]
        POS_at_50_pop += res[4]
        POS_at_100_pop += res[5]
        NEG_at_1_pop += res[6]
        NEG_at_5_pop += res[7]
        NEG_at_10_pop += res[8]
        NEG_at_20_pop += res[9]
        NEG_at_50_pop += res[10]
        NEG_at_100_pop += res[11]
        users_DEL_pop += res[12]
        users_INS_pop += res[13]
        rank_at_1_pop += res[14]
        rank_at_5_pop += res[15]
        rank_at_10_A_pop += res[16]
        rank_at_20_A_pop += res[17]
        rank_at_50_A_pop += res[18]
        rank_at_100_A_pop += res[19]
        rank_at_k_B_pop += res[20]
        NDCG_at_1_pop += res[21]
        NDCG_at_5_pop += res[22]
        NDCG_at_10_pop += res[23]
        NDCG_at_20_pop += res[24]
        NDCG_at_50_pop += res[25]
        NDCG_at_100_pop += res[26]

        ### lime:
        res = single_user_metrics(user_vector, item_id, num_of_bins, num_items, VAE_recommender, modelCombined_g, mask_type= 'lime')    
        POS_at_1_lime += res[0]
        pos_at_5_lime += res[1]
        POS_at_10_lime += res[2]
        POS_at_20_lime += res[3]
        POS_at_50_lime += res[4]
        POS_at_100_lime += res[5]
        NEG_at_1_lime += res[6]
        NEG_at_5_lime += res[7]
        NEG_at_10_lime += res[8]
        NEG_at_20_lime += res[9]
        NEG_at_50_lime += res[10]
        NEG_at_100_lime += res[11]
        users_DEL_lime += res[12]
        users_INS_lime += res[13]
        rank_at_1_lime += res[14]
        rank_at_5_lime += res[15]
        rank_at_10_A_lime += res[16]
        rank_at_20_A_lime += res[17]
        rank_at_50_A_lime += res[18]
        rank_at_100_A_lime += res[19]
        rank_at_k_B_lime += res[20]
        NDCG_at_1_lime += res[21]
        NDCG_at_5_lime += res[22]
        NDCG_at_10_lime += res[23]
        NDCG_at_20_lime += res[24]
        NDCG_at_50_lime += res[25]
        NDCG_at_100_lime += res[26]

        
        res = single_user_metrics(user_vector, item_id, num_of_bins, num_items, VAE_recommender, modelCombined_g, mask_type= 'tf_idf')    
        POS_at_1_tf_idf += res[0]
        pos_at_5_tf_idf += res[1]
        POS_at_10_tf_idf += res[2]
        POS_at_20_tf_idf += res[3]
        POS_at_50_tf_idf += res[4]
        POS_at_100_tf_idf += res[5]
        NEG_at_1_tf_idf += res[6]
        NEG_at_5_tf_idf += res[7]
        NEG_at_10_tf_idf += res[8]
        NEG_at_20_tf_idf += res[9]
        NEG_at_50_tf_idf += res[10]
        NEG_at_100_tf_idf += res[11]
        users_DEL_tf_idf += res[12]
        users_INS_tf_idf += res[13]
        rank_at_1_tf_idf += res[14]
        rank_at_5_tf_idf += res[15]
        rank_at_10_A_tf_idf += res[16]
        rank_at_20_A_tf_idf += res[17]
        rank_at_50_A_tf_idf += res[18]
        rank_at_100_A_tf_idf += res[19]
        rank_at_k_B_tf_idf += res[20]
        NDCG_at_1_tf_idf += res[21]
        NDCG_at_5_tf_idf += res[22]
        NDCG_at_10_tf_idf += res[23]
        NDCG_at_20_tf_idf += res[24]
        NDCG_at_50_tf_idf += res[25]
        NDCG_at_100_tf_idf += res[26]
        
        ## SHAP
        res = single_user_metrics(user_vector,item_id, num_of_bins, num_items, VAE_recommender, modelCombined_g,  user_id =user_id, mask_type= 'shap')    
    
        POS_at_1_shap += res[0]
        pos_at_5_shap += res[1]
        POS_at_10_shap += res[2]
        POS_at_20_shap += res[3]
        POS_at_50_shap += res[4]
        POS_at_100_shap += res[5]
        NEG_at_1_shap += res[6]
        NEG_at_5_shap += res[7]
        NEG_at_10_shap += res[8]
        NEG_at_20_shap += res[9]
        NEG_at_50_shap += res[10]
        NEG_at_100_shap += res[11]
        users_DEL_shap += res[12]
        users_INS_shap += res[13]
        rank_at_1_shap += res[14]
        rank_at_5_shap += res[15]
        rank_at_10_A_shap += res[16]
        rank_at_20_A_shap += res[17]
        rank_at_50_A_shap += res[18]
        rank_at_100_A_shap += res[19]
        rank_at_k_B_shap += res[20]
        NDCG_at_1_shap += res[21]
        NDCG_at_5_shap += res[22]
        NDCG_at_10_shap += res[23]
        NDCG_at_20_shap += res[24]
        NDCG_at_50_shap += res[25]
        NDCG_at_100_shap += res[26]

        ### lxr:
        res = single_user_metrics(user_vector, item_id, num_of_bins, num_items, VAE_recommender, modelCombined_g, mask_type= 'lxr')    
        POS_at_1_lxr += res[0]
        pos_at_5_lxr += res[1]
        POS_at_10_lxr += res[2]
        POS_at_20_lxr += res[3]
        POS_at_50_lxr += res[4]
        POS_at_100_lxr += res[5]
        NEG_at_1_lxr += res[6]
        NEG_at_5_lxr += res[7]
        NEG_at_10_lxr += res[8]
        NEG_at_20_lxr += res[9]
        NEG_at_50_lxr += res[10]
        NEG_at_100_lxr += res[11]
        users_DEL_lxr += res[12]
        users_INS_lxr += res[13]
        rank_at_1_lxr += res[14]
        rank_at_5_lxr += res[15]
        rank_at_10_A_lxr += res[16]
        rank_at_20_A_lxr += res[17]
        rank_at_50_A_lxr += res[18]
        rank_at_100_A_lxr += res[19]
        rank_at_k_B_lxr += res[20]
        NDCG_at_1_lxr += res[21]
        NDCG_at_5_lxr += res[22]
        NDCG_at_10_lxr += res[23]
        NDCG_at_20_lxr += res[24]
        NDCG_at_50_lxr += res[25]
        NDCG_at_100_lxr += res[26]

        prev_time = time.time()
        print("User {}, total time: {:.2f}".format(i,prev_time - start_time))
    a = i+1

print('POS_at_1_j_u: ', np.mean(POS_at_1_j_u[1:])/a)
print('pos_at_5_j_u: ', np.mean(pos_at_5_j_u[1:])/a)
print('POS_at_10_j_u: ', np.mean(POS_at_10_j_u[1:])/a)
print('POS_at_20_j_u: ', np.mean(POS_at_20_j_u[1:])/a)
print('POS_at_50_j_u: ', np.mean(POS_at_50_j_u[1:])/a)
print('POS_at_100_j_u: ', np.mean(POS_at_100_j_u[1:])/a)
print('NEG_at_1_j_u: ', np.mean(NEG_at_1_j_u[1:])/a)
print('NEG_at_5_j_u: ', np.mean(NEG_at_5_j_u[1:])/a)
print('NEG_at_10_j_u: ', np.mean(NEG_at_10_j_u[1:])/a)
print('NEG_at_20_j_u: ', np.mean(NEG_at_20_j_u[1:])/a)
print('NEG_at_50_j_u: ', np.mean(NEG_at_50_j_u[1:])/a)
print('NEG_at_100_j_u: ', np.mean(NEG_at_100_j_u[1:])/a)
print('users_DEL_j_u: ', np.mean(users_DEL_j_u[1:])/a)
print('users_INS_j_u: ', np.mean(users_INS_j_u[1:])/a)
print('rank_at_1_j_u: ', np.mean(rank_at_1_j_u[1:])/a)
print('rank_at_5_j_u: ', np.mean(rank_at_5_j_u[1:])/a)
print('rank_at_10_A_j_u: ', np.mean(rank_at_10_A_j_u[1:])/a)
print('rank_at_20_A_j_u: ', np.mean(rank_at_20_A_j_u[1:])/a)
print('rank_at_50_A_j_u: ', np.mean(rank_at_50_A_j_u[1:])/a)
print('rank_at_100_A_j_u: ', np.mean(rank_at_100_A_j_u[1:])/a)
print('rank_at_k_B_j_u: ', np.mean(rank_at_k_B_j_u[1:])/a)
print('NDCG_at_1_j_u: ', np.mean(NDCG_at_1_j_u[1:])/a)
print('NDCG_at_5_j_u: ', np.mean(NDCG_at_5_j_u[1:])/a)
print('NDCG_at_10_j_u: ', np.mean(NDCG_at_10_j_u[1:])/a)
print('NDCG_at_20_j_u: ', np.mean(NDCG_at_20_j_u[1:])/a)
print('NDCG_at_50_j_u: ', np.mean(NDCG_at_50_j_u[1:])/a)
print('NDCG_at_100_j_u: ', np.mean(NDCG_at_100_j_u[1:])/a)

print('POS_at_1_j_g: ', np.mean(POS_at_1_j_g[1:])/a)
print('pos_at_5_j_g: ', np.mean(pos_at_5_j_g[1:])/a)
print('POS_at_10_j_g: ', np.mean(POS_at_10_j_g[1:])/a)
print('POS_at_20_j_g: ', np.mean(POS_at_20_j_g[1:])/a)
print('POS_at_50_j_g: ', np.mean(POS_at_50_j_g[1:])/a)
print('POS_at_100_j_g: ', np.mean(POS_at_100_j_g[1:])/a)
print('NEG_at_1_j_g: ', np.mean(NEG_at_1_j_g[1:])/a)
print('NEG_at_5_j_g: ', np.mean(NEG_at_5_j_g[1:])/a)
print('NEG_at_10_j_g: ', np.mean(NEG_at_10_j_g[1:])/a)
print('NEG_at_20_j_g: ', np.mean(NEG_at_20_j_g[1:])/a)
print('NEG_at_50_j_g: ', np.mean(NEG_at_50_j_g[1:])/a)
print('NEG_at_100_j_g: ', np.mean(NEG_at_100_j_g[1:])/a)
print('users_DEL_j_g: ', np.mean(users_DEL_j_g[1:])/a)
print('users_INS_j_g: ', np.mean(users_INS_j_g[1:])/a)
print('rank_at_1_j_g: ', np.mean(rank_at_1_j_g[1:])/a)
print('rank_at_5_j_g: ', np.mean(rank_at_5_j_g[1:])/a)
print('rank_at_10_A_j_g: ', np.mean(rank_at_10_A_j_g[1:])/a)
print('rank_at_20_A_j_g: ', np.mean(rank_at_20_A_j_g[1:])/a)
print('rank_at_50_A_j_g: ', np.mean(rank_at_50_A_j_g[1:])/a)
print('rank_at_100_A_j_g: ', np.mean(rank_at_100_A_j_g[1:])/a)
print('rank_at_k_B_j_g: ', np.mean(rank_at_k_B_j_g[1:])/a)
print('NDCG_at_1_j_g: ', np.mean(NDCG_at_1_j_g[1:])/a)
print('NDCG_at_5_j_g: ', np.mean(NDCG_at_5_j_g[1:])/a)
print('NDCG_at_10_j_g: ', np.mean(NDCG_at_10_j_g[1:])/a)
print('NDCG_at_20_j_g: ', np.mean(NDCG_at_20_j_g[1:])/a)
print('NDCG_at_50_j_g: ', np.mean(NDCG_at_50_j_g[1:])/a)
print('NDCG_at_100_j_g: ', np.mean(NDCG_at_100_j_g[1:])/a)

print('POS_at_1_c_s: ', np.mean(POS_at_1_c_s[1:])/a)
print('pos_at_5_c_s: ', np.mean(pos_at_5_c_s[1:])/a)
print('POS_at_10_c_s: ', np.mean(POS_at_10_c_s[1:])/a)
print('POS_at_20_c_s: ', np.mean(POS_at_20_c_s[1:])/a)
print('POS_at_50_c_s: ', np.mean(POS_at_50_c_s[1:])/a)
print('POS_at_100_c_s: ', np.mean(POS_at_100_c_s[1:])/a)
print('NEG_at_1_c_s: ', np.mean(NEG_at_1_c_s[1:])/a)
print('NEG_at_5_c_s: ', np.mean(NEG_at_5_c_s[1:])/a)
print('NEG_at_10_c_s: ', np.mean(NEG_at_10_c_s[1:])/a)
print('NEG_at_20_c_s: ', np.mean(NEG_at_20_c_s[1:])/a)
print('NEG_at_50_c_s: ', np.mean(NEG_at_50_c_s[1:])/a)
print('NEG_at_100_c_s: ', np.mean(NEG_at_100_c_s[1:])/a)
print('users_DEL_c_s: ', np.mean(users_DEL_c_s[1:])/a)
print('users_INS_c_s: ', np.mean(users_INS_c_s[1:])/a)
print('rank_at_1_c_s: ', np.mean(rank_at_1_c_s[1:])/a)
print('rank_at_5_c_s: ', np.mean(rank_at_5_c_s[1:])/a)
print('rank_at_10_A_c_s: ', np.mean(rank_at_10_A_c_s[1:])/a)
print('rank_at_20_A_c_s: ', np.mean(rank_at_20_A_c_s[1:])/a)
print('rank_at_50_A_c_s: ', np.mean(rank_at_50_A_c_s[1:])/a)
print('rank_at_100_A_c_s: ', np.mean(rank_at_100_A_c_s[1:])/a)
print('rank_at_k_B_c_s: ', np.mean(rank_at_k_B_c_s[1:])/a)
print('NDCG_at_1_c_s: ', np.mean(NDCG_at_1_c_s[1:])/a)
print('NDCG_at_5_c_s: ', np.mean(NDCG_at_5_c_s[1:])/a)
print('NDCG_at_10_c_s: ', np.mean(NDCG_at_10_c_s[1:])/a)
print('NDCG_at_20_c_s: ', np.mean(NDCG_at_20_c_s[1:])/a)
print('NDCG_at_50_c_s: ', np.mean(NDCG_at_50_c_s[1:])/a)
print('NDCG_at_100_c_s: ', np.mean(NDCG_at_100_c_s[1:])/a)

print('POS_at_1_pop: ', np.mean(POS_at_1_pop[1:])/a)
print('pos_at_5_pop: ', np.mean(pos_at_5_pop[1:])/a)
print('POS_at_10_pop: ', np.mean(POS_at_10_pop[1:])/a)
print('POS_at_20_pop: ', np.mean(POS_at_20_pop[1:])/a)
print('POS_at_50_pop: ', np.mean(POS_at_50_pop[1:])/a)
print('POS_at_100_pop: ', np.mean(POS_at_100_pop[1:])/a)
print('NEG_at_1_pop: ', np.mean(NEG_at_1_pop[1:])/a)
print('NEG_at_5_pop: ', np.mean(NEG_at_5_pop[1:])/a)
print('NEG_at_10_pop: ', np.mean(NEG_at_10_pop[1:])/a)
print('NEG_at_20_pop: ', np.mean(NEG_at_20_pop[1:])/a)
print('NEG_at_50_pop: ', np.mean(NEG_at_50_pop[1:])/a)
print('NEG_at_100_pop: ', np.mean(NEG_at_100_pop[1:])/a)
print('users_DEL_pop: ', np.mean(users_DEL_pop[1:])/a)
print('users_INS_pop: ', np.mean(users_INS_pop[1:])/a)
print('rank_at_1_pop: ', np.mean(rank_at_1_pop[1:])/a)
print('rank_at_5_pop: ', np.mean(rank_at_5_pop[1:])/a)
print('rank_at_10_A_pop: ', np.mean(rank_at_10_A_pop[1:])/a)
print('rank_at_20_A_pop: ', np.mean(rank_at_20_A_pop[1:])/a)
print('rank_at_50_A_pop: ', np.mean(rank_at_50_A_pop[1:])/a)
print('rank_at_100_A_pop: ', np.mean(rank_at_100_A_pop[1:])/a)
print('rank_at_k_B_pop: ', np.mean(rank_at_k_B_pop[1:])/a)
print('NDCG_at_1_pop: ', np.mean(NDCG_at_1_pop[1:])/a)
print('NDCG_at_5_pop: ', np.mean(NDCG_at_5_pop[1:])/a)
print('NDCG_at_10_pop: ', np.mean(NDCG_at_10_pop[1:])/a)
print('NDCG_at_20_pop: ', np.mean(NDCG_at_20_pop[1:])/a)
print('NDCG_at_50_pop: ', np.mean(NDCG_at_50_pop[1:])/a)
print('NDCG_at_100_pop: ', np.mean(NDCG_at_100_pop[1:])/a)

print('POS_at_1_lime: ', np.mean(POS_at_1_lime[1:])/a)
print('pos_at_5_lime: ', np.mean(pos_at_5_lime[1:])/a)
print('POS_at_10_lime: ', np.mean(POS_at_10_lime[1:])/a)
print('POS_at_20_lime: ', np.mean(POS_at_20_lime[1:])/a)
print('POS_at_50_lime: ', np.mean(POS_at_50_lime[1:])/a)
print('POS_at_100_lime: ', np.mean(POS_at_100_lime[1:])/a)
print('NEG_at_1_lime: ', np.mean(NEG_at_1_lime[1:])/a)
print('NEG_at_5_lime: ', np.mean(NEG_at_5_lime[1:])/a)
print('NEG_at_10_lime: ', np.mean(NEG_at_10_lime[1:])/a)
print('NEG_at_20_lime: ', np.mean(NEG_at_20_lime[1:])/a)
print('NEG_at_50_lime: ', np.mean(NEG_at_50_lime[1:])/a)
print('NEG_at_100_lime: ', np.mean(NEG_at_100_lime[1:])/a)
print('users_DEL_lime: ', np.mean(users_DEL_lime[1:])/a)
print('users_INS_lime: ', np.mean(users_INS_lime[1:])/a)
print('rank_at_1_lime: ', np.mean(rank_at_1_lime[1:])/a)
print('rank_at_5_lime: ', np.mean(rank_at_5_lime[1:])/a)
print('rank_at_10_A_lime: ', np.mean(rank_at_10_A_lime[1:])/a)
print('rank_at_20_A_lime: ', np.mean(rank_at_20_A_lime[1:])/a)
print('rank_at_50_A_lime: ', np.mean(rank_at_50_A_lime[1:])/a)
print('rank_at_100_A_lime: ', np.mean(rank_at_100_A_lime[1:])/a)
print('rank_at_k_B_lime: ', np.mean(rank_at_k_B_lime[1:])/a)
print('NDCG_at_1_lime: ', np.mean(NDCG_at_1_lime[1:])/a)
print('NDCG_at_5_lime: ', np.mean(NDCG_at_5_lime[1:])/a)
print('NDCG_at_10_lime: ', np.mean(NDCG_at_10_lime[1:])/a)
print('NDCG_at_20_lime: ', np.mean(NDCG_at_20_lime[1:])/a)
print('NDCG_at_50_lime: ', np.mean(NDCG_at_50_lime[1:])/a)
print('NDCG_at_100_lime: ', np.mean(NDCG_at_100_lime[1:])/a)

print('POS_at_1_tf_idf: ', np.mean(POS_at_1_tf_idf[1:])/a)
print('pos_at_5_tf_idf: ', np.mean(pos_at_5_tf_idf[1:])/a)
print('POS_at_10_tf_idf: ', np.mean(POS_at_10_tf_idf[1:])/a)
print('POS_at_20_tf_idf: ', np.mean(POS_at_20_tf_idf[1:])/a)
print('POS_at_50_tf_idf: ', np.mean(POS_at_50_tf_idf[1:])/a)
print('POS_at_100_tf_idf: ', np.mean(POS_at_100_tf_idf[1:])/a)
print('NEG_at_1_tf_idf: ', np.mean(NEG_at_1_tf_idf[1:])/a)
print('NEG_at_5_tf_idf: ', np.mean(NEG_at_5_tf_idf[1:])/a)
print('NEG_at_10_tf_idf: ', np.mean(NEG_at_10_tf_idf[1:])/a)
print('NEG_at_20_tf_idf: ', np.mean(NEG_at_20_tf_idf[1:])/a)
print('NEG_at_50_tf_idf: ', np.mean(NEG_at_50_tf_idf[1:])/a)
print('NEG_at_100_tf_idf: ', np.mean(NEG_at_100_tf_idf[1:])/a)
print('users_DEL_tf_idf: ', np.mean(users_DEL_tf_idf[1:])/a)
print('users_INS_tf_idf: ', np.mean(users_INS_tf_idf[1:])/a)
print('rank_at_1_tf_idf: ', np.mean(rank_at_1_tf_idf[1:])/a)
print('rank_at_5_tf_idf: ', np.mean(rank_at_5_tf_idf[1:])/a)
print('rank_at_10_A_tf_idf: ', np.mean(rank_at_10_A_tf_idf[1:])/a)
print('rank_at_20_A_tf_idf: ', np.mean(rank_at_20_A_tf_idf[1:])/a)
print('rank_at_50_A_tf_idf: ', np.mean(rank_at_50_A_tf_idf[1:])/a)
print('rank_at_100_A_tf_idf: ', np.mean(rank_at_100_A_tf_idf[1:])/a)
print('rank_at_k_B_tf_idf: ', np.mean(rank_at_k_B_tf_idf[1:])/a)
print('NDCG_at_1_tf_idf: ', np.mean(NDCG_at_1_tf_idf[1:])/a)
print('NDCG_at_5_tf_idf: ', np.mean(NDCG_at_5_tf_idf[1:])/a)
print('NDCG_at_10_tf_idf: ', np.mean(NDCG_at_10_tf_idf[1:])/a)
print('NDCG_at_20_tf_idf: ', np.mean(NDCG_at_20_tf_idf[1:])/a)
print('NDCG_at_50_tf_idf: ', np.mean(NDCG_at_50_tf_idf[1:])/a)
print('NDCG_at_100_tf_idf: ', np.mean(NDCG_at_100_tf_idf[1:])/a)

print('POS_at_1_shap: ', np.mean(POS_at_1_shap[1:])/a)
print('POS_at_5_shap: ', np.mean(pos_at_5_shap[1:])/a)
print('POS_at_10_shap: ', np.mean(POS_at_10_shap[1:])/a)
print('POS_at_20_shap: ', np.mean(POS_at_20_shap[1:])/a)
print('POS_at_50_shap: ', np.mean(POS_at_50_shap[1:])/a)
print('POS_at_100_shap: ', np.mean(POS_at_100_shap[1:])/a)
print('NEG_at_1_shap: ', np.mean(NEG_at_1_shap[1:])/a)
print('NEG_at_5_shap: ', np.mean(NEG_at_5_shap[1:])/a)
print('NEG_at_10_shap: ', np.mean(NEG_at_10_shap[1:])/a)
print('NEG_at_20_shap: ', np.mean(NEG_at_20_shap[1:])/a)
print('NEG_at_50_shap: ', np.mean(NEG_at_50_shap[1:])/a)
print('NEG_at_100_shap: ', np.mean(NEG_at_100_shap[1:])/a)
print('users_DEL_shap: ', np.mean(users_DEL_shap[1:])/a)
print('users_INS_shap: ', np.mean(users_INS_shap[1:])/a)
print('rank_at_1_shap: ', np.mean(rank_at_1_shap[1:])/a)
print('rank_at_5_shap: ', np.mean(rank_at_5_shap[1:])/a)
print('rank_at_10_A_shap: ', np.mean(rank_at_10_A_shap[1:])/a)
print('rank_at_20_A_shap: ', np.mean(rank_at_20_A_shap[1:])/a)
print('rank_at_50_A_shap: ', np.mean(rank_at_50_A_shap[1:])/a)
print('rank_at_100_A_shap: ', np.mean(rank_at_100_A_shap[1:])/a)
print('rank_at_k_B_shap: ', np.mean(rank_at_k_B_shap[1:])/a)
print('NDCG_at_1_shap: ', np.mean(NDCG_at_1_shap[1:])/a)
print('NDCG_at_5_shap: ', np.mean(NDCG_at_5_shap[1:])/a)
print('NDCG_at_10_shap: ', np.mean(NDCG_at_10_shap[1:])/a)
print('NDCG_at_20_shap: ', np.mean(NDCG_at_20_shap[1:])/a)
print('NDCG_at_50_shap: ', np.mean(NDCG_at_50_shap[1:])/a)
print('NDCG_at_100_shap: ', np.mean(NDCG_at_100_shap[1:])/a)

print('POS_at_1_lxr: ', np.mean(POS_at_1_lxr[1:])/a)
print('pos_at_5_lxr: ', np.mean(pos_at_5_lxr[1:])/a)
print('POS_at_10_lxr: ', np.mean(POS_at_10_lxr[1:])/a)
print('POS_at_20_lxr: ', np.mean(POS_at_20_lxr[1:])/a)
print('POS_at_50_lxr: ', np.mean(POS_at_50_lxr[1:])/a)
print('POS_at_100_lxr: ', np.mean(POS_at_100_lxr[1:])/a)
print('NEG_at_1_lxr: ', np.mean(NEG_at_1_lxr[1:])/a)
print('NEG_at_5_lxr: ', np.mean(NEG_at_5_lxr[1:])/a)
print('NEG_at_10_lxr: ', np.mean(NEG_at_10_lxr[1:])/a)
print('NEG_at_20_lxr: ', np.mean(NEG_at_20_lxr[1:])/a)
print('NEG_at_50_lxr: ', np.mean(NEG_at_50_lxr[1:])/a)
print('NEG_at_100_lxr: ', np.mean(NEG_at_100_lxr[1:])/a)
print('users_DEL_lxr: ', np.mean(users_DEL_lxr[1:])/a)
print('users_INS_lxr: ', np.mean(users_INS_lxr[1:])/a)
print('rank_at_1_lxr: ', np.mean(rank_at_1_lxr[1:])/a)
print('rank_at_5_lxr: ', np.mean(rank_at_5_lxr[1:])/a)
print('rank_at_10_A_lxr: ', np.mean(rank_at_10_A_lxr[1:])/a)
print('rank_at_20_A_lxr: ', np.mean(rank_at_20_A_lxr[1:])/a)
print('rank_at_50_A_lxr: ', np.mean(rank_at_50_A_lxr[1:])/a)
print('rank_at_100_A_lxr: ', np.mean(rank_at_100_A_lxr[1:])/a)
print('rank_at_k_B_lxr: ', np.mean(rank_at_k_B_lxr[1:])/a)
print('NDCG_at_1_lxr: ', np.mean(NDCG_at_1_lxr[1:])/a)
print('NDCG_at_5_lxr: ', np.mean(NDCG_at_5_lxr[1:])/a)
print('NDCG_at_10_lxr: ', np.mean(NDCG_at_10_lxr[1:])/a)
print('NDCG_at_20_lxr: ', np.mean(NDCG_at_20_lxr[1:])/a)
print('NDCG_at_50_lxr: ', np.mean(NDCG_at_50_lxr[1:])/a)
print('NDCG_at_100_lxr: ', np.mean(NDCG_at_100_lxr[1:])/a)

In [ ]:
full_df = pd.DataFrame(columns = ['names', 'values'])

In [ ]:
names = ['POS_at_1_j_u', 'pos_at_5_j_u', 'POS_at_10_j_u', 'POS_at_20_j_u', 'POS_at_50_j_u', 'POS_at_100_j_u', 'NEG_at_1_j_u', 'NEG_at_5_j_u', 'NEG_at_10_j_u', 'NEG_at_20_j_u', 'NEG_at_50_j_u', 'NEG_at_100_j_u', 'users_DEL_j_u', 'users_INS_j_u', 'rank_at_1_j_u', 'rank_at_5_j_u', 'rank_at_10_A_j_u', 'rank_at_20_A_j_u', 'rank_at_50_A_j_u', 'rank_at_100_A_j_u', 'rank_at_k_B_j_u', 'NDCG_at_1_j_u', 'NDCG_at_5_j_u', 'NDCG_at_10_j_u', 'NDCG_at_20_j_u', 'NDCG_at_50_j_u', 'NDCG_at_100_j_u']
values = [POS_at_1_j_u, pos_at_5_j_u, POS_at_10_j_u, POS_at_20_j_u, POS_at_50_j_u, POS_at_100_j_u, NEG_at_1_j_u, NEG_at_5_j_u, NEG_at_10_j_u, NEG_at_20_j_u, NEG_at_50_j_u, NEG_at_100_j_u, users_DEL_j_u, users_INS_j_u, rank_at_1_j_u, rank_at_5_j_u, rank_at_10_A_j_u, rank_at_20_A_j_u, rank_at_50_A_j_u, rank_at_100_A_j_u, rank_at_k_B_j_u, NDCG_at_1_j_u, NDCG_at_5_j_u, NDCG_at_10_j_u, NDCG_at_20_j_u, NDCG_at_50_j_u, NDCG_at_100_j_u]

d = {'names':names, 'values': values}

temp_df = pd.DataFrame(d)


normalized_values = []
for i in range(len(values)):
    lst = []
    for j in range(len(values[i])):
        lst.append(values[i][j]/a)
    normalized_values.append(lst)

temp_df['normalized_values'] = normalized_values

AUC = []
for i in range(len(values)):
    sublist = temp_df['normalized_values'][i][1:]
    AUC.append(np.mean(sublist))

temp_df['AUC'] = AUC

full_df = pd.concat([full_df, temp_df], ignore_index = True)


############################################################################################


names = ['POS_at_1_j_g', 'pos_at_5_j_g', 'POS_at_10_j_g', 'POS_at_20_j_g', 'POS_at_50_j_g', 'POS_at_100_j_g', 'NEG_at_1_j_g', 'NEG_at_5_j_g', 'NEG_at_10_j_g', 'NEG_at_20_j_g', 'NEG_at_50_j_g', 'NEG_at_100_j_g', 'users_DEL_j_g', 'users_INS_j_g', 'rank_at_1_j_g', 'rank_at_5_j_g', 'rank_at_10_A_j_g', 'rank_at_20_A_j_g', 'rank_at_50_A_j_g', 'rank_at_100_A_j_g', 'rank_at_k_B_j_g', 'NDCG_at_1_j_g', 'NDCG_at_5_j_g', 'NDCG_at_10_j_g', 'NDCG_at_20_j_g', 'NDCG_at_50_j_g', 'NDCG_at_100_j_g']
values = [POS_at_1_j_g, pos_at_5_j_g, POS_at_10_j_g, POS_at_20_j_g, POS_at_50_j_g, POS_at_100_j_g, NEG_at_1_j_g, NEG_at_5_j_g, NEG_at_10_j_g, NEG_at_20_j_g, NEG_at_50_j_g, NEG_at_100_j_g, users_DEL_j_g, users_INS_j_g, rank_at_1_j_g, rank_at_5_j_g, rank_at_10_A_j_g, rank_at_20_A_j_g, rank_at_50_A_j_g, rank_at_100_A_j_g, rank_at_k_B_j_g, NDCG_at_1_j_g, NDCG_at_5_j_g, NDCG_at_10_j_g, NDCG_at_20_j_g, NDCG_at_50_j_g, NDCG_at_100_j_g]

d = {'names':names, 'values': values}

temp_df = pd.DataFrame(d)


normalized_values = []
for i in range(len(values)):
    lst = []
    for j in range(len(values[i])):
        lst.append(values[i][j]/a)
    normalized_values.append(lst)

temp_df['normalized_values'] = normalized_values

AUC = []
for i in range(len(values)):
    sublist = temp_df['normalized_values'][i][1:]
    AUC.append(np.mean(sublist))

temp_df['AUC'] = AUC

full_df = pd.concat([full_df, temp_df], ignore_index = True)


############################################################################################


names = ['POS_at_1_c_s', 'pos_at_5_c_s', 'POS_at_10_c_s', 'POS_at_20_c_s', 'POS_at_50_c_s', 'POS_at_100_c_s', 'NEG_at_1_c_s', 'NEG_at_5_c_s', 'NEG_at_10_c_s', 'NEG_at_20_c_s', 'NEG_at_50_c_s', 'NEG_at_100_c_s', 'users_DEL_c_s', 'users_INS_c_s', 'rank_at_1_c_s', 'rank_at_5_c_s', 'rank_at_10_A_c_s', 'rank_at_20_A_c_s', 'rank_at_50_A_c_s', 'rank_at_100_A_c_s', 'rank_at_k_B_c_s', 'NDCG_at_1_c_s', 'NDCG_at_5_c_s', 'NDCG_at_10_c_s', 'NDCG_at_20_c_s', 'NDCG_at_50_c_s', 'NDCG_at_100_c_s']
values = [POS_at_1_c_s, pos_at_5_c_s, POS_at_10_c_s, POS_at_20_c_s, POS_at_50_c_s, POS_at_100_c_s, NEG_at_1_c_s, NEG_at_5_c_s, NEG_at_10_c_s, NEG_at_20_c_s, NEG_at_50_c_s, NEG_at_100_c_s, users_DEL_c_s, users_INS_c_s, rank_at_1_c_s, rank_at_5_c_s, rank_at_10_A_c_s, rank_at_20_A_c_s, rank_at_50_A_c_s, rank_at_100_A_c_s, rank_at_k_B_c_s, NDCG_at_1_c_s, NDCG_at_5_c_s, NDCG_at_10_c_s, NDCG_at_20_c_s, NDCG_at_50_c_s, NDCG_at_100_c_s]

d = {'names':names, 'values': values}

temp_df = pd.DataFrame(d)


normalized_values = []
for i in range(len(values)):
    lst = []
    for j in range(len(values[i])):
        lst.append(values[i][j]/a)
    normalized_values.append(lst)

temp_df['normalized_values'] = normalized_values

AUC = []
for i in range(len(values)):
    sublist = temp_df['normalized_values'][i][1:]
    AUC.append(np.mean(sublist))

temp_df['AUC'] = AUC

full_df = pd.concat([full_df, temp_df], ignore_index = True)


############################################################################################


names = ['POS_at_1_pop', 'pos_at_5_pop', 'POS_at_10_pop', 'POS_at_20_pop', 'POS_at_50_pop', 'POS_at_100_pop', 'NEG_at_1_pop', 'NEG_at_5_pop', 'NEG_at_10_pop', 'NEG_at_20_pop', 'NEG_at_50_pop', 'NEG_at_100_pop', 'users_DEL_pop', 'users_INS_pop', 'rank_at_1_pop', 'rank_at_5_pop', 'rank_at_10_A_pop', 'rank_at_20_A_pop', 'rank_at_50_A_pop', 'rank_at_100_A_pop', 'rank_at_k_B_pop', 'NDCG_at_1_pop', 'NDCG_at_5_pop', 'NDCG_at_10_pop', 'NDCG_at_20_pop', 'NDCG_at_50_pop', 'NDCG_at_100_pop']
values = [POS_at_1_pop, pos_at_5_pop, POS_at_10_pop, POS_at_20_pop, POS_at_50_pop, POS_at_100_pop, NEG_at_1_pop, NEG_at_5_pop, NEG_at_10_pop, NEG_at_20_pop, NEG_at_50_pop, NEG_at_100_pop, users_DEL_pop, users_INS_pop, rank_at_1_pop, rank_at_5_pop, rank_at_10_A_pop, rank_at_20_A_pop, rank_at_50_A_pop, rank_at_100_A_pop, rank_at_k_B_pop, NDCG_at_1_pop, NDCG_at_5_pop, NDCG_at_10_pop, NDCG_at_20_pop, NDCG_at_50_pop, NDCG_at_100_pop]

d = {'names':names, 'values': values}

temp_df = pd.DataFrame(d)


normalized_values = []
for i in range(len(values)):
    lst = []
    for j in range(len(values[i])):
        lst.append(values[i][j]/a)
    normalized_values.append(lst)

temp_df['normalized_values'] = normalized_values

AUC = []
for i in range(len(values)):
    sublist = temp_df['normalized_values'][i][1:]
    AUC.append(np.mean(sublist))

temp_df['AUC'] = AUC

full_df = pd.concat([full_df, temp_df], ignore_index = True)


############################################################################################


names = ['POS_at_1_lime', 'pos_at_5_lime', 'POS_at_10_lime', 'POS_at_20_lime', 'POS_at_50_lime', 'POS_at_100_lime', 'NEG_at_1_lime', 'NEG_at_5_lime', 'NEG_at_10_lime', 'NEG_at_20_lime', 'NEG_at_50_lime', 'NEG_at_100_lime', 'users_DEL_lime', 'users_INS_lime', 'rank_at_1_lime', 'rank_at_5_lime', 'rank_at_10_A_lime', 'rank_at_20_A_lime', 'rank_at_50_A_lime', 'rank_at_100_A_lime', 'rank_at_k_B_lime', 'NDCG_at_1_lime', 'NDCG_at_5_lime', 'NDCG_at_10_lime', 'NDCG_at_20_lime', 'NDCG_at_50_lime', 'NDCG_at_100_lime']
values = [POS_at_1_lime, pos_at_5_lime, POS_at_10_lime, POS_at_20_lime, POS_at_50_lime, POS_at_100_lime, NEG_at_1_lime, NEG_at_5_lime, NEG_at_10_lime, NEG_at_20_lime, NEG_at_50_lime, NEG_at_100_lime, users_DEL_lime, users_INS_lime, rank_at_1_lime, rank_at_5_lime, rank_at_10_A_lime, rank_at_20_A_lime, rank_at_50_A_lime, rank_at_100_A_lime, rank_at_k_B_lime, NDCG_at_1_lime, NDCG_at_5_lime, NDCG_at_10_lime, NDCG_at_20_lime, NDCG_at_50_lime, NDCG_at_100_lime]

d = {'names':names, 'values': values}

temp_df = pd.DataFrame(d)


normalized_values = []
for i in range(len(values)):
    lst = []
    for j in range(len(values[i])):
        lst.append(values[i][j]/a)
    normalized_values.append(lst)

temp_df['normalized_values'] = normalized_values

AUC = []
for i in range(len(values)):
    sublist = temp_df['normalized_values'][i][1:]
    AUC.append(np.mean(sublist))

temp_df['AUC'] = AUC

full_df = pd.concat([full_df, temp_df], ignore_index = True)


############################################################################################


names = ['POS_at_1_tf_idf', 'pos_at_5_tf_idf', 'POS_at_10_tf_idf', 'POS_at_20_tf_idf', 'POS_at_50_tf_idf', 'POS_at_100_tf_idf', 'NEG_at_1_tf_idf', 'NEG_at_5_tf_idf', 'NEG_at_10_tf_idf', 'NEG_at_20_tf_idf', 'NEG_at_50_tf_idf', 'NEG_at_100_tf_idf', 'users_DEL_tf_idf', 'users_INS_tf_idf', 'rank_at_1_tf_idf', 'rank_at_5_tf_idf', 'rank_at_10_A_tf_idf', 'rank_at_20_A_tf_idf', 'rank_at_50_A_tf_idf', 'rank_at_100_A_tf_idf', 'rank_at_k_B_tf_idf', 'NDCG_at_1_tf_idf', 'NDCG_at_5_tf_idf', 'NDCG_at_10_tf_idf', 'NDCG_at_20_tf_idf', 'NDCG_at_50_tf_idf', 'NDCG_at_100_tf_idf']
values = [POS_at_1_tf_idf, pos_at_5_tf_idf, POS_at_10_tf_idf, POS_at_20_tf_idf, POS_at_50_tf_idf, POS_at_100_tf_idf, NEG_at_1_tf_idf, NEG_at_5_tf_idf, NEG_at_10_tf_idf, NEG_at_20_tf_idf, NEG_at_50_tf_idf, NEG_at_100_tf_idf, users_DEL_tf_idf, users_INS_tf_idf, rank_at_1_tf_idf, rank_at_5_tf_idf, rank_at_10_A_tf_idf, rank_at_20_A_tf_idf, rank_at_50_A_tf_idf, rank_at_100_A_tf_idf, rank_at_k_B_tf_idf, NDCG_at_1_tf_idf, NDCG_at_5_tf_idf, NDCG_at_10_tf_idf, NDCG_at_20_tf_idf, NDCG_at_50_tf_idf, NDCG_at_100_tf_idf]

d = {'names':names, 'values': values}

temp_df = pd.DataFrame(d)


normalized_values = []
for i in range(len(values)):
    lst = []
    for j in range(len(values[i])):
        lst.append(values[i][j]/a)
    normalized_values.append(lst)

temp_df['normalized_values'] = normalized_values

AUC = []
for i in range(len(values)):
    sublist = temp_df['normalized_values'][i][1:]
    AUC.append(np.mean(sublist))

temp_df['AUC'] = AUC

full_df = pd.concat([full_df, temp_df], ignore_index = True)


############################################################################################

names = ['POS_at_1_shap', 'pos_at_5_shap', 'POS_at_10_shap', 'POS_at_20_shap', 'POS_at_50_shap', 'POS_at_100_shap', 'NEG_at_1_shap', 'NEG_at_5_shap', 'NEG_at_10_shap', 'NEG_at_20_shap', 'NEG_at_50_shap', 'NEG_at_100_shap', 'users_DEL_shap', 'users_INS_shap', 'rank_at_1_shap', 'rank_at_5_shap', 'rank_at_10_A_shap', 'rank_at_20_A_shap', 'rank_at_50_A_shap', 'rank_at_100_A_shap', 'rank_at_k_B_shap', 'NDCG_at_1_shap', 'NDCG_at_5_shap', 'NDCG_at_10_shap', 'NDCG_at_20_shap', 'NDCG_at_50_shap', 'NDCG_at_100_shap']
values = [POS_at_1_shap, pos_at_5_shap, POS_at_10_shap, POS_at_20_shap, POS_at_50_shap, POS_at_100_shap, NEG_at_1_shap, NEG_at_5_shap, NEG_at_10_shap, NEG_at_20_shap, NEG_at_50_shap, NEG_at_100_shap, users_DEL_shap, users_INS_shap, rank_at_1_shap, rank_at_5_shap, rank_at_10_A_shap, rank_at_20_A_shap, rank_at_50_A_shap, rank_at_100_A_shap, rank_at_k_B_shap, NDCG_at_1_shap, NDCG_at_5_shap, NDCG_at_10_shap, NDCG_at_20_shap, NDCG_at_50_shap, NDCG_at_100_shap]

d = {'names':names, 'values': values}

temp_df = pd.DataFrame(d)


normalized_values = []
for i in range(len(values)):
    lst = []
    for j in range(len(values[i])):
        lst.append(values[i][j]/a)
    normalized_values.append(lst)

temp_df['normalized_values'] = normalized_values

AUC = []
for i in range(len(values)):
    sublist = temp_df['normalized_values'][i][1:]
    AUC.append(np.mean(sublist))

temp_df['AUC'] = AUC

full_df = pd.concat([full_df, temp_df], ignore_index = True)


############################################################################################

names = ['POS_at_1_lxr', 'pos_at_5_lxr', 'POS_at_10_lxr', 'POS_at_20_lxr', 'POS_at_50_lxr', 'POS_at_100_lxr', 'NEG_at_1_lxr', 'NEG_at_5_lxr', 'NEG_at_10_lxr', 'NEG_at_20_lxr', 'NEG_at_50_lxr', 'NEG_at_100_lxr', 'users_DEL_lxr', 'users_INS_lxr', 'rank_at_1_lxr', 'rank_at_5_lxr', 'rank_at_10_A_lxr', 'rank_at_20_A_lxr', 'rank_at_50_A_lxr', 'rank_at_100_A_lxr', 'rank_at_k_B_lxr', 'NDCG_at_1_lxr', 'NDCG_at_5_lxr', 'NDCG_at_10_lxr', 'NDCG_at_20_lxr', 'NDCG_at_50_lxr', 'NDCG_at_100_lxr']
values = [POS_at_1_lxr, pos_at_5_lxr, POS_at_10_lxr, POS_at_20_lxr, POS_at_50_lxr, POS_at_100_lxr, NEG_at_1_lxr, NEG_at_5_lxr, NEG_at_10_lxr, NEG_at_20_lxr, NEG_at_50_lxr, NEG_at_100_lxr, users_DEL_lxr, users_INS_lxr, rank_at_1_lxr, rank_at_5_lxr, rank_at_10_A_lxr, rank_at_20_A_lxr, rank_at_50_A_lxr, rank_at_100_A_lxr, rank_at_k_B_lxr, NDCG_at_1_lxr, NDCG_at_5_lxr, NDCG_at_10_lxr, NDCG_at_20_lxr, NDCG_at_50_lxr, NDCG_at_100_lxr]

d = {'names':names, 'values': values}

temp_df = pd.DataFrame(d)


normalized_values = []
for i in range(len(values)):
    lst = []
    for j in range(len(values[i])):
        lst.append(values[i][j]/a)
    normalized_values.append(lst)

temp_df['normalized_values'] = normalized_values

AUC = []
for i in range(len(values)):
    sublist = temp_df['normalized_values'][i][1:]
    AUC.append(np.mean(sublist))

temp_df['AUC'] = AUC

full_df = pd.concat([full_df, temp_df], ignore_index = True)